In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
def process_company(company_data):
    try:
        m = company_data.money_raised_usd.values
        p = company_data.post_money_valuation_usd.values
        e = company_data.exit_value.values[0]

        if company_data.exit_type.values[0] == 3:
            ret = np.zeros_like(company_data.uuid) - 1
        elif company_data.shape[0] == 1:
            ret = [(m[0] / p[0] * e - m[0]) / m[0]]
        else:
            x0 = m / p # stake of investors at their first round
            x = (p[1:] - m[1:]) / p[1:] # dilution factors
            d = np.cumprod(x)[::-1]
            d = np.insert(d, len(d), 1)
            ret = (e * x0 * d - m) / m
        return pd.DataFrame(data=ret, index=company_data.uuid)
    except:
        print(company_data)
        return pd.DataFrame(data=[np.nan] * company_data.uuid.size, index=company_data.uuid)


def filter_dates(funding_round):
    return funding_round['announced_on'] < funding_round['exit_date']

In [16]:
predicted = True

In [3]:
# rounds_data = pd.read_csv('../../data/obs_predicted_data.csv')
# rounds_data = pd.read_csv('../../data/no_pred_data.csv')
rounds_data = pd.read_csv('../../data/early_data.csv')

In [4]:
rounds_data

,uuid,org_uuid,announced_on,money_raised_usd,num_investors,investment_type,lead_investor_rank,post_money_valuation_usd,prev_rounds,country_code,amount_delta,holding_time
0,773c9fb9-0e27-4115-8bc1-343e24ee28c8,0002aa63-f21b-4c54-9495-895a1d09e0d4,2946326400,1500000.0,7.0,seed,206226.0,4.500000e+06,0.0,USA,1500000.0,498.0
1,e5e06be5-4b9a-4040-9ee4-bb45920a1f99,00060741-9e4b-44cd-880a-8d0fdd8046c8,2969827200,215983.0,2.0,pre_seed,96497.0,2.215982e+06,0.0,SGP,215983.0,40.0
2,23263b3b-0c61-4980-aee2-47f00f12e357,000b7667-bf70-4298-bae9-94f44d27dc10,2966716800,8700000.0,1.0,seed,3151372.0,1.970000e+07,0.0,CHN,8700000.0,157.0
3,befaba3e-2bae-4fd5-b71e-6d03a71b980a,0010d353-dfe4-483b-86a7-5e11e9996e63,2887401600,500000.0,1.0,pre_seed,215643.0,4.500000e+06,1.0,USA,200000.0,122.0
4,10e2bc7d-f5a6-4cdd-acf3-6c87b167f83f,0011daf2-bf81-2365-e08a-d896bdc186d6,2902262400,1900000.0,1.0,series_a,245.0,6.400000e+06,1.0,IND,1795000.0,622.0
...,...,...,...,...,...,...,...,...,...,...,...,...
92206,4ed08f61-2001-4458-a29e-a651bb1e18be,fffe3b0b-8545-4414-9c0e-a3f961c4e420,2874960000,14523274.0,1.0,other,295238.0,7.536181e+07,0.0,CHN,14523274.0,1450.0
92207,b6cd618e-c9f0-4410-9f5e-4243b2b65fcd,ffff2bea-7e44-4563-b311-65daadb683cd,2680214400,451407.0,1.0,angel,2819467.0,1.701184e+06,0.0,CHN,451407.0,207.0
92208,f5cac82f-41b8-4992-abee-ac609fad4e3f,ffff2bea-7e44-4563-b311-65daadb683cd,2755382400,1616916.0,1.0,series_a,3439.0,7.957348e+06,1.0,CHN,1165509.0,870.0
92209,d5d0d173-9a6f-4359-f4bb-a7e71f20b988,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,2851804800,210000.0,4.0,seed,43.0,2.487609e+06,0.0,USA,210000.0,500.0


In [5]:
orgs = pd.read_csv("../../data/crunchbase/organizations.csv")
acqs = pd.read_csv("../../data/crunchbase/acquisitions.csv")
ipos = pd.read_csv("../../data/crunchbase/ipos.csv")

In [6]:
ipos = ipos[['org_uuid', 'went_public_on', 'valuation_price_usd']]
acqs = acqs[['acquiree_uuid', 'acquired_on', 'price_usd']]

In [7]:
rounds = rounds_data[['uuid', 'org_uuid', 'announced_on', 'money_raised_usd', 'post_money_valuation_usd']].copy()

ipos.columns = ['org_uuid', 'exit_date', 'exit_value']
acqs.columns = ['org_uuid', 'exit_date', 'exit_value']

clos = pd.DataFrame(orgs[orgs.status == 'closed'][['uuid', 'closed_on']])
clos.columns = ['org_uuid', 'exit_date']

ipos["exit_type"] = 1
acqs["exit_type"] = 2
clos["exit_type"] = 3
clos["exit_value"] = np.nan

exits = pd.concat([acqs, ipos, clos]).set_index("org_uuid")

In [8]:
rounds["exit_date"] = pd.to_datetime(rounds.org_uuid.map(exits["exit_date"].to_dict()))
rounds["exit_type"] = rounds.org_uuid.map(exits["exit_type"].to_dict()).fillna(value=4).astype(int)
rounds["exit_value"] = rounds.org_uuid.map(exits["exit_value"].to_dict())

In [9]:
ref_date = '1926-01-01'
rounds.announced_on = pd.Timestamp(ref_date) + rounds.announced_on * pd.Timedelta('1s')
rounds = rounds[(rounds.announced_on < rounds.exit_date)]
rounds = rounds.sort_values(by=['org_uuid', 'announced_on'])

In [10]:
rounds['return_usd'] = rounds.uuid.map(rounds.groupby('org_uuid').apply(process_company).droplevel(0)[0]) + 1

In [11]:
rounds = rounds.drop(columns=['uuid'])
rounds.insert(1, 'group_num', np.nan)
rounds.insert(2, 'seg_num', np.nan)

rounds.columns = ['company_num', 'group_num', 'seg_num', 'round_date', 'raised_usd',
                  'postvalue_usd', 'exit_date', 'exit_type', 'exit_value', 'return_usd']
rounds = rounds[rounds.round_date > '1990-01-01']

In [12]:
operating = rounds_data[['org_uuid', 'announced_on', 'money_raised_usd', 'post_money_valuation_usd']].copy()
operating = operating[~operating.org_uuid.isin(rounds.company_num)]

operating.announced_on = pd.Timestamp(ref_date) + operating.announced_on * pd.Timedelta('1s')
operating.insert(1, 'group_num', np.nan)
operating.insert(2, 'seg_num', np.nan)
operating.insert(operating.shape[1], 'exit_date', np.nan)
operating.insert(operating.shape[1], 'exit_type', 4)
operating.insert(operating.shape[1], 'exit_value', np.nan)
operating.insert(operating.shape[1], 'return_usd', np.nan)

operating.columns = ['company_num', 'group_num', 'seg_num', 'round_date', 'raised_usd',
                  'postvalue_usd', 'exit_date', 'exit_type', 'exit_value', 'return_usd']
operating = operating[operating.round_date.dt.year > 1990]

In [13]:
complete = pd.concat([rounds, operating], ignore_index=True)
complete.loc[complete.return_usd == 0, 'return_usd'] = np.nan
complete = complete[(complete.return_usd < 3000) | (complete.return_usd.isna())]
complete.exit_date = pd.to_datetime(complete.exit_date, errors="coerce")

In [14]:
# complete.to_csv('../../data/data.csv', index=False)
# complete.to_csv('../../data/data_nopred_full.csv', index=False)
complete.to_csv('../../data/data_early_full.csv', index=False)